In [2]:
import pyrosetta

In [3]:
pyrosetta.init('-out:level 300')

PyRosetta-4 2021 [Rosetta PyRosetta4.conda.linux.cxx11thread.serialization.CentOS.python38.Release 2021.50+release.4ff291ed8257ec2d0cd4d96ae4289e1d39ce1007 2021-12-16T00:25:15] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.conda.linux.cxx11thread.serialization.CentOS.python38.Release r306 2021.50+release.4ff291e 4ff291ed8257ec2d0cd4d96ae4289e1d39ce1007 http://www.pyrosetta.org 2021-12-16T00:25:15
core.init: command: PyRosetta -out:level 300 -database /projects/crispy_shifty/envs/crispy/lib/python3.8/site-packages/pyrosetta/database
basic.random.init_random_generator: 'RNG device' seed mode, using '/dev/urandom', seed=1313007469 seed_offset=0 real_seed=1313007469 thread_index=0
basic.random.init_random_generator: RandomGenerator:init: Normal mode, seed=1313007469 RG_type=mt19937


In [4]:
import bz2
with open('/home/broerman/projects/crispy_shifty/projects/crispy_shifty_dimers/03_close_loops/decoys/0000/03_close_loops_6a49b6ffb2c140768271417169250e44.pdb.bz2', 'rb') as f:
    looped_pose = pyrosetta.rosetta.core.pose.Pose()
    pyrosetta.rosetta.core.import_pose.pose_from_pdbstring(looped_pose, bz2.decompress(f.read()).decode())
with open('/home/broerman/projects/crispy_shifty/projects/crispy_shifty_dimers/02_design_filter_3/decoys/0003/_d3d15b52f1e24b6a8366a884a6650849.pdb.bz2', 'rb') as f:
    designed_pose = pyrosetta.rosetta.core.pose.Pose()
    pyrosetta.rosetta.core.import_pose.pose_from_pdbstring(designed_pose, bz2.decompress(f.read()).decode())
with open('/home/broerman/projects/crispy_shifty/projects/crispy_shifty_dimers/01_make_states/decoys/0001/CSD_01_make_states_47e5e5b5c2c7456daf00eed5ceb58cf0.pdb.bz2', 'rb') as f:
    docked_pose = pyrosetta.rosetta.core.pose.Pose()
    pyrosetta.rosetta.core.import_pose.pose_from_pdbstring(docked_pose, bz2.decompress(f.read()).decode())

core.chemical.GlobalResidueTypeSet: Finished initializing fa_standard residue type set.  Created 983 residue types
core.chemical.GlobalResidueTypeSet: Total time to initialize 0.904227 seconds.
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OXT on residue GLN:CtermProteinFull 80
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OXT on residue GLN:CtermProteinFull 242


In [5]:
import sys
sys.path.insert(0, "/mnt/home/broerman/projects/crispy_shifty")
from crispy_shifty.protocols.design import interface_among_chains
from crispy_shifty.utils.io import pymol_selection
design_sel = interface_among_chains(chain_list=[1, 2, 3, 4], vector_mode=True)
print(pymol_selection(designed_pose, design_sel, "designed"))
design_loop_sel = pyrosetta.rosetta.core.select.residue_selector.ResidueIndexSelector(
    ','.join(
        [str(x) for x in 
            pyrosetta.rosetta.core.select.get_residues_from_subset(
                pyrosetta.rosetta.core.select.residue_selector.AndResidueSelector(
                    design_sel, 
                    pyrosetta.rosetta.core.select.residue_selector.ChainSelector(1)
                ).apply(designed_pose)
            )
        ] + \
        [str(x+2) for x in 
            pyrosetta.rosetta.core.select.get_residues_from_subset(
                pyrosetta.rosetta.core.select.residue_selector.AndResidueSelector(
                    design_sel, 
                    pyrosetta.rosetta.core.select.residue_selector.OrResidueSelector(
                        pyrosetta.rosetta.core.select.residue_selector.ChainSelector(2),
                        pyrosetta.rosetta.core.select.residue_selector.ChainSelector(3),
                    )
                ).apply(designed_pose)
            )
        ] + \
        [str(x+4) for x in 
            pyrosetta.rosetta.core.select.get_residues_from_subset(
                pyrosetta.rosetta.core.select.residue_selector.AndResidueSelector(
                    design_sel, 
                    pyrosetta.rosetta.core.select.residue_selector.ChainSelector(4)
                ).apply(designed_pose)
            )
        ]
    )
)

print(pymol_selection(looped_pose, design_loop_sel, "designed_loop"))

select designed, (chain A and resid 1,2,5,9,24,27,28,31,42,43,44,46,47,49,50,51,53,54,57,61,63,64,65,66,68,69,70,72,73,75,76,77,79,80) or (chain B and resid 84,87,88,91,92,94,95,98,103,106,107,110,111,113,114,117,118) or (chain C and resid 164,167,205,206,208,209,212,213,215,216,219,220,223,225,228,231,232,234,235,238,239,241,242) or (chain D and resid 244,245,246,247,249,250,251,253,254,256,257,258,260,261,262,265,266,268,269,270,272,273,275,276,277,279,280,282,288,291,292,295,298,299,302,317,321)
select designed_loop, (chain A and resid 1,2,5,9,24,27,28,31,42,43,44,46,47,49,50,51,53,54,57,61,63,64,65,66,68,69,70,72,73,75,76,77,79,80,86,89,90,93,94,96,97,100,105,108,109,112,113,115,116,119,120) or (chain B and resid 166,169,207,208,210,211,214,215,217,218,221,222,225,227,230,233,234,236,237,240,241,243,244,248,249,250,251,253,254,255,257,258,260,261,262,264,265,266,269,270,272,273,274,276,277,279,280,281,283,284,286,292,295,296,299,302,303,306,321,325)
